In [1]:
import os
import sys
import torch
import random
import numpy as np
import matplotlib.pyplot as plt

In [3]:
path = "../data/mill19/rubble-pixsfm/train/data_partitions"
files = [f for f in os.listdir(path) if f.endswith('.npy')]
print(files)

['block_rubble_all_lr_c12_loss_10_r4.npy', 'block_rubble_all_lr_c9_loss_10_r4.npy', 'block_rubble_all_lr_c4_loss_10_r4.npy', 'block_rubble_all_lr_c25_loss_10_r4.npy', 'block_rubble_all_lr_c9_loss_12_r4.npy', 'block_rubble_all_lr_c9_loss_14_r4.npy']


In [10]:
distributions = []
for f in files:
    partition = np.load(os.path.join(path, f))
    distribution = partition.sum(axis=0)
    mean = int(distribution.mean())
    std = int(distribution.std())
    distributions.append(partition.sum(axis=0))
    print(f, mean, std)

block_rubble_all_lr_c12_loss_10_r4.npy 432 124
block_rubble_all_lr_c9_loss_10_r4.npy 552 128
block_rubble_all_lr_c4_loss_10_r4.npy 835 68
block_rubble_all_lr_c25_loss_10_r4.npy 355 115
block_rubble_all_lr_c9_loss_12_r4.npy 469 145
block_rubble_all_lr_c9_loss_14_r4.npy 415 148
